In [ ]:
import pandas as pd
df=pd.read_csv("House_Price.csv")
df.head()

In [ ]:
print(df.info())
print(df.describe())

In [ ]:
import seaborn as sns
import matplotlib as plt
print(sns.jointplot(data=df,x="n_hos_beds",y="price",space=0.001))
print(sns.jointplot(data=df,x="n_hos_beds",y="price",space=1))

In [ ]:
sns.lineplot(data=df,x="n_hot_rooms",y="price",hue="waterbody")

In [ ]:
help(sns.lineplot)

In [ ]:
print(sns.countplot(x="airport",data=df))
print(sns.countplot(x="waterbody",data=df))

In [ ]:
import numpy as np
uv=np.percentile(df.n_hot_rooms,[99])[0]
df[(df.n_hot_rooms>uv)]

In [ ]:
df.n_hot_rooms[(df.n_hot_rooms>3*uv)]=3*uv

In [ ]:
sns.jointplot(x="n_hot_rooms",y="price",data=df)

In [ ]:
lv=np.percentile(df.rainfall,[1])[0]
df[df.rainfall<lv]

In [ ]:
df.rainfall[(df.rainfall<0.3*lv)]=0.3*lv

In [ ]:
df.info()

In [ ]:
df.n_hos_beds=df.n_hos_beds.fillna(df.n_hos_beds.mean())


In [ ]:
sns.jointplot(data=df,x="crime_rate",y="price")

In [ ]:
#df.crime_rate=np.log(1+df.crime_rate)

In [ ]:
df['avg_dist']=(df.dist1+df.dist2+df.dist3+df.dist4)/4


In [ ]:
sns.jointplot(data=df,x="avg_dist",y="price")

In [ ]:
df.avg_dist=np.exp(df.avg_dist-1)


In [ ]:
sns.jointplot(data=df,x="avg_dist",y="price")

In [ ]:
df.corr()

In [ ]:
del df['bus_ter']

In [ ]:
df=pd.get_dummies(df)
df.head(10)

In [ ]:
del df['airport_NO']


In [ ]:
del df['waterbody_None']

In [ ]:
#del df['bus_ter_YES']
df.head()

In [ ]:
#Linear_Regression
import statsmodels.api as sn
x=sn.add_constant(df['room_num'])
lm=sn.OLS(df['price'],x).fit()
lm.summary()

In [ ]:
from sklearn.linear_model import LinearRegression
Y=df['price']
X=df[['room_num']]
lm2=LinearRegression()
lm2.fit(X,Y)
print(lm2.coef_,lm2.intercept_)

In [ ]:
sns.jointplot(x=df['room_num'],y=df['price'],data=df)

In [ ]:
xx1=sn.add_constant(df['n_hot_rooms'])
lm5=sn.OLS(df['price'],xx1).fit();
lm5.summary()

In [ ]:
sns.jointplot(data=df,y=df['price'],x=df['n_hot_rooms'],kind='reg')

In [ ]:
x=sn.add_constant(df['crime_rate'])
lm4=sn.OLS(df['price'],x).fit()
lm4.summary()

In [ ]:
x=df[['price']]
y=df['crime_rate']
lm3=LinearRegression()
lm3.fit(x,y)
print(lm3.coef_,lm3.intercept_)

In [ ]:
sns.jointplot(x=df['crime_rate'],y=df['price'],data=df,kind='reg')

In [ ]:
#Multiple liner regression
x_multi=df.drop('price',axis=1)
y_multi=df['price']
x_multi_cons=sn.add_constant(x_multi)
x_multi_cons.head()

In [ ]:
lm_mul=sn.OLS(y_multi,x_multi_cons).fit()
lm_mul.summary()

In [ ]:
lm_mul1=LinearRegression()
lm_mul1.fit(x_multi,y_multi)
print(lm_mul1.intercept_,lm_mul1.coef_)

In [ ]:
#Test_Train_test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_multi,y_multi,test_size=0.2,random_state=0)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
lm_a=LinearRegression()
lm_a.fit(x_train,y_train)
y_test_a=lm_a.predict(x_test)
y_train_a=lm_a.predict(x_train)


In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_test_a)

In [ ]:
r2_score(y_train,y_train_a)

In [ ]:
#standardizing the data
from sklearn import preprocessing
scaler=preprocessing.StandardScaler().fit(x_train)
x_train_s=scaler.transform(x_train)
x_test_s=scaler.transform(x_test)

In [ ]:
#Ridge regression
from sklearn.linear_model import Ridge
lm_r=Ridge(alpha=0.5)
lm_r.fit(x_train_s,y_train)
r2_score(y_test,lm_r.predict(x_test_s))

In [ ]:
#finding the best value of lambda in Ridge

In [ ]:
from sklearn.model_selection import validation_curve
param_rang=np.logspace(-2,8,100)
param_rang

In [ ]:
train_score, test_score=validation_curve(Ridge(),x_train_s,y_train,param_name="alpha",param_range=param_rang,scoring="r2",cv=3)
#?validation_curve

In [ ]:
print(test_score)
print(train_score)

In [ ]:
train_mean=np.mean(train_score,axis=1)
test_mean=np.mean(test_score,axis=1)
train_mean
max(test_mean)

In [ ]:
sns.jointplot(x=np.log(param_rang),y=test_mean)

In [ ]:
np.where(test_mean==max(test_mean))

In [ ]:
param_rang[34]

In [ ]:
lm_r_best=Ridge(alpha=param_rang[34])
lm_r_best.fit(x_train_s,y_train)
r2_score(y_test,lm_r_best.predict(x_test_s))

In [ ]:
r2_score(y_train,lm_r_best.predict(x_train_s))